# Working with CRS: Reprojecting and Transforming Vector Data
### by [Kate Vavra-Musser](https://vavramusser.github.io) for the [R Spatial Notebook Series](https://vavramusser.github.io/r-spatial)

## Introduction
[Coordinate Reference Systems (CRS)](https://en.wikipedia.org/wiki/Spatial_reference_system) play a foundational role in spatial data analysis. They define how data based on the three-dimensional surface of the earth can be represented in flat, two-dimensional maps. Understanding CRS is essential for working with spatial data, especially if you are working with datasets from different sources that may use different CRSs.  If you want to map more than one spatial data layer on the same map, or if you want to use multiple spatial data files in the same spatial analysis workflow, you will need to ensure all layers are in the same CRS.

### Geographic vs Projected CRS
There are two main types of CRSs: Geographic CRS and Projected CRS.

#### Geographic CRS (GCS)
A Geographic CRS represents locations using **latitude and longitude** on a three-dimensional sphere or ellipsoid.  Geographic CRS are best for storing and displaying global data, but are not ideal for area measurements due to local distortions.

**When to Use Geographic CRS**
* When dealing with global datasets (e.g., satellite data, GPS data).
* When storing and sharing data (most GIS databases store raw data in a geographic CRS).
* For web maps (WGS 84 is the most common for web-based applications).

**Drawbacks**
* Not suitable for measuring distances or areas accurately, as degrees of latitude and longitude do not have uniform lengths across the globe.

#### Projected Coordinate Reference Systems (PCS)
A Projected CRS transforms the Earth's curved surface onto a flat, two-dimensional plane, allowing for accurate distance, area, and direction measurements.  Projected CRS use linear units (e.g. meters or feet) instead of degrees.  They are designed to minimize distortion for a **specific area** and are best for regional and local spatial analysis where accurate distance and area calculations are needed.

**When to Use Projected CRS**
* For regional and local analysis where distance and area measurements matter (e.g., urban planning, land surveys).
* When performing spatial operations like buffering or area calculations.
* For maps that need to preserve distances, shapes, or areas accurately.

**Drawbacks**
* Only accurate within its defined area.  A projected CRS optimized for North America won’t work well for Europe.
* May distort some aspects of geometry, depending on the projection type.

#### Common CRS
| **CRS Name** | **Type** | **Common Use Case** | **EPSG Code**
|:-|:-|:-|:-
| [World Geodetic System 1984 (WGS 84)](https://en.wikipedia.org/wiki/World_Geodetic_System) | Geographic | Global standard, used in GPS and web maps | [4326](https://epsg.io/4326) |
| [Web Mercator (WGS 84)](https://en.wikipedia.org/wiki/Web_Mercator_projection) | Projected | Used in web maps (Google Maps, OSM, Bing) | [3857](https://epsg.io/3857) |
| [North American Datum 1983 (NAD83)](https://en.wikipedia.org/wiki/North_American_Datum) | Geographic | North America-based datasets | [4269](https://epsg.io/4269) |
| [Albers Equal Area  Conic (NAD83)](https://en.wikipedia.org/wiki/Albers_projection) | Projected | US Census, socio-economic studies | [5070](https://epsg.io/5070) |

### Tips for Choosing and Working with CRS

* When mapping multiple layers or carrying out analyses that use multiple datasets, all layers must share the same CRS.
* Certain projections minimize distortions in distance, area, or angles, which is critical for tasks like distance measurements or area calculations.
* Some map projections are better suited for specific regions or visualization purposes.
* No projection is perfect!  Choose one that best suits your analysis needs.
* Always verify that the CRS information is correctly assigned, especially when working with new or imported datasets.
* Keep track of transformations in your workflow for reproducibility.

### Notebook Goals
In this notebook, you will explore how to work with CRSs in R using the sf package including reprojecting spatial data between different CRSs.

### ✨ Prerequisites ✨
* Complete [Introduction to sf: Reading, Writing, and Inspecting Vector Data](https://platform.i-guide.io/notebooks/9968babe-22e4-4c3d-98e2-d8b45e9672cd)

### 💽 Data Used in this Notebook 💽
* United States Boundary Shapefile (*usa_boundary.shp*)
  * You can also download a copy of *usa_boundary.zip* file from [the I-GUIDE platform](https://platform.i-guide.io/datasets/aae279db-71f0-47e4-91e7-e8ac8791ea56) or directly from [Kate's GitHub](https://github.com/vavramusser/r-spatial/blob/main/usa_boundary.zip).  You will need to unzip *usa_boundary.zip* and extract *usa_boundary.shp* file to your workspace.

### Notebook Overview
1. Setup
2. Reprojecting and Transforming Spatial Data

---

## 1. Setup
This notebook requires the following R packages and functions.

[**sf**](https://cran.r-project.org/web/packages/sf/index.html) · Support for [simple features](https://r-spatial.github.io/sf/articles/sf1.html), a standardized way to encode spatial vector data - Binds to [*GDAL*](https://gdal.org/en/stable) for reading and writing data, to [*GEOS*](https://libgeos.org) for geometrical operations, and to [*PROJ*](https://proj.org/en/stable) for projection conversions and datum transformations - Uses by default the [*s2*](https://cran.r-project.org/web/packages/s2/index.html) package for spherical geometry operations on ellipsoidal (long/lat) coordinates · This notebook uses the following functions from *sf*.

* [*st_crs*](https://rdrr.io/cran/sf/man/st_crs.html) · retrieve coordinate reference system from object
* [*st_geometry*](https://rdrr.io/cran/sf/man/st_geometry.html) · get, set, replace or rename geometry from an sf object
* [*st_read*](https://rdrr.io/cran/sf/man/st_read.html) · read simple features or layers from file or database
* [*st_transform*](https://rdrr.io/cran/sf/man/st_transform.html) · transform or convert coordinates of simple feature

### 1a. Install and Load Required Packages
If you have not already installed the required packages, uncomment and run the code below:

In [2]:
# install.packages(c("sf"))

Load the packages into your workspace.

In [3]:
library(sf)

Linking to GEOS 3.11.2, GDAL 3.8.2, PROJ 9.3.1; sf_use_s2() is TRUE



### 1b. Read in the Example Data

In [4]:
# read the shapefile into an sf object
usa_boundary <- st_read("usa_boundary.shp")

Reading layer `usa_boundary' from data source 
  `C:\Users\vavra\Dropbox\R Spatial\r-spatial\usa_boundary.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 166 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -171.7911 ymin: 18.91619 xmax: -66.96466 ymax: 71.35776
Geodetic CRS:  WGS 84


## 2. Reprojecting and Transforming Spatial Data

### 2a. Identifying CRS with [*st_crs*](https://rdrr.io/cran/sf/man/st_crs.html)

The [*st_crs*](https://rdrr.io/cran/sf/man/st_crs.html) function from the [**sf**](https://cran.r-project.org/web/packages/sf/index.html) package retrieves information about the CRS of an sf object. It can also be used to assign a CRS to data.

In [ ]:
# View the CRS of the dataset
st_crs(usa_boundary)

The output includes details like the EPSG code and projection definition (Proj4 string or WKT).

In [ ]:
plot(st_geometry(usa_boundary))

### 2b. Reprojecting Data with [*st_transform*](https://rdrr.io/cran/sf/man/st_transform.html)

The [*st_transform*](https://rdrr.io/cran/sf/man/st_transform.html) function from the [**sf**](https://cran.r-project.org/web/packages/sf/index.html) package changes the CRS of spatial data. This is useful when aligning data from different sources or preparing data for analyses that require a specific projection.

In [ ]:
# Transform to USA Contiguous Albers Equal Area Conic CRS (NAD83) using EPSG
usa_boundary_albers <- st_transform(usa_boundary, crs = 5070)

# Check the CRS of the transformed dataset
st_crs(usa_boundary_albers)

In [ ]:
par(mfrow = c(2, 1))
plot(st_geometry(usa_boundary), main = "WGS 1984")
plot(st_geometry(usa_boundary_albers), main = "Albers Equal Area Conic")

#### Example: Transforming to a Custom CRS

You can also specify a CRS using a WKT string or Proj4 string:

In [ ]:
# Transform to US National Atlas Equal Area CRS (NAD83) using Proj.4 string
usa_boundary_atlas <- st_transform(usa_boundary, crs = "+proj=laea +lat_0=45 +lon_0=-100 +x_0=0 +y_0=0 +ellps=sphere +units=m +no_defs +type=crs")

In [ ]:
par(mfrow = c(3, 1))
plot(st_geometry(usa_boundary), main = "WGS 1984")
plot(st_geometry(usa_boundary_albers), main = "Albers Equal Area Conic")
plot(st_geometry(usa_boundary_atlas), main = "Atlas Equal Area")

#### Example: Transforming Using Another Object's CRS
Finally, you can also directly pass the CRS of another object.

In [ ]:
# Transform to WGS 1984 using the CRS of the original usa_boundary file
usa_boundary_wgs <- st_transform(usa_boundary_albers, crs = st_crs(usa_boundary))

In [ ]:
par(mfrow = c(2, 2))
plot(st_geometry(usa_boundary), main = "WGS 1984")
plot(st_geometry(usa_boundary_albers), main = "Albers Equal Area Conic")
plot(st_geometry(usa_boundary_atlas), main = "Atlas Equal Area")
plot(st_geometry(usa_boundary_wgs), main = "WGS 1984")

## Review

This notebook provided an introduction to working with CRSs in R using the [*sf*](https://cran.r-project.org/web/packages/sf/index.html) package, including:

* Inspecting CRSs with [*st_crs*](https://rdrr.io/cran/sf/man/st_crs.html)
* Transforming data with [*st_transform*](https://rdrr.io/cran/sf/man/st_transform.html)

---

## Next Steps

* Continue to [**Chapter 2.03 Preparing Vector Data for Analysis**](https://platform.i-guide.io/notebooks/44926d85-7f08-4774-a103-a22ff3876cad)
* Move on to Chapter 3: IPUMS Data Acquisition and Extraction
  * [**Chapter 3.01 IPUMS USA Data Extraction using ipumsr**](https://platform.i-guide.io/notebooks/ab5cad39-6d00-43d2-bc51-17fd4e6b98f2)
  * [**Chapter 3.02 IPUMS NHGIS Data Extraction using ipumsr**](https://platform.i-guide.io/notebooks/be08e56e-1c08-458e-a230-263c64d386bc)
* Move on to Chapter 4: Open-Source GIS Data Acquisition and Extraction
  * [**Chapter 4.01 Importing Data from Natural Earth with rnaturalearth and rnaturalearthdata**]()
* Return to the [**R Spatial Notebooks Project Chapter List**](https://vavramusser.github.io/r-spatial/#:~:text=Chapter%201%3A%20Data%20Sources%20and%20APIs) to view a list of all available notebooks organized in the R Spatial Notebooks chapter structure.
* Visit the [**R Spatial Notebooks Project Homepage**](https://vavramusser.github.io/r-spatial) to learn more about the project, view the list of all notebooks, and explore additional resources.
* Join the project [**Mailing List**](https://mailchi.mp/ab01e8fc8397/r-spatial-email-signup) to hear about future notebook releases and other updates.
* If you have an idea for a new notebook please submit your idea via the [**Suggestion Box**](https://us19.list-manage.com/survey?u=746bf8d366d6fbc99c699e714&id=54590a28ea&attribution=false).

---

## ★ Thank You ★

Thank you so much for engaging with this notebook and supporting the project!  The R Spatial Notebooks Project is a labor of love so if you enjoy or benefit from these notebooks, please consider [**Donating to the Project**](https://buymeacoffee.com/vavramusser).  Your support allows me to continue producing notebooks and supporting the R Spatial Notebooks community.